In [ ]:
import openai
import random
import pandas as pd
from tqdm import tqdm
import numpy as np
from sklearn.metrics import f1_score,accuracy_score
from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Chem import DataStructs
from rdkit.Chem import rdMolDescriptors
from rdkit import Chem
import warnings
from rdkit import RDLogger
from steamship import Steamship
import datetime
import os 
from openai import OpenAI
os.environ["OPENAI_API_KEY"] = ""
openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


Data preprocessing

In [32]:
# random.seed(42) 
# #read bace dataset
# bace = pd.read_csv("BACE.csv")
# sample_size = 500
# bace_sample= bace.sample(sample_size)
# bace.drop(bace_sample.index, inplace = True)

In [33]:
##save sampled dataset

# bace_sample.to_csv("BACE_test.csv",index = False)
# bace.to_csv("BACE_train.csv",index =False)
# print(bace_sample['Class'].value_counts())

Class
0    289
1    211
Name: count, dtype: int64


In [19]:
def generate_response_by_gpt4(prompt, model_engine = "gpt-4o"):
    completion = client.chat.completions.create(
        model=model_engine, temperature=1, n=5, 
        messages=[{"role": "user", "content": prompt}],
    )

    message = completion.choices
    message = [i.message.content.strip() for i in message]
    return message

def generate_response_by_gpt35(prompt, model_engine = "gpt-3.5-turbo"):
    completion = client.chat.completions.create(
        model=model_engine, temperature=1, n=5, 
        messages=[{"role": "user", "content": prompt}],
    )
    message = completion.choices
    message = [i.message.content.strip() for i in message]
    return message

Sampling strategy: 1. random sample, 2.scaffold sample


In [ ]:
# random sampling

from tdc.single_pred import ADME
from tdc.single_pred import Tox
## CYP3A4_Substrate_CarbonMangels
data = ADME(name = 'BBB_Martins')
split = data.get_split()
name ='bbb' #hERG
#data = Tox(name = name)
split = data.get_split()
bbb_train = split['train']
bbb_test = split['test']
bbb_test.rename(columns={'Y': 'Class'}, inplace=True)
bbb_test.rename(columns={'Drug': 'mol'}, inplace=True)
bbb_train.rename(columns={'Y': 'Class'}, inplace=True)
bbb_train.rename(columns={'Drug': 'mol'}, inplace=True)


def generate_response_by_gpt4(prompt, model_engine = "gpt-4o"):
    completion = client.chat.completions.create(
        model=model_engine, temperature=1, n=5, 
        messages=[{"role": "user", "content": prompt}],
    )

    message = completion.choices
    message = [i.message.content.strip() for i in message]
    return message

def radom_sample_examples(data,sample_size):
    positive_examples = data[data["Class"] == 1].sample(int(sample_size/2))
    negative_examples = data[data["Class"] == 0].sample(int(sample_size/2))
    smiles = positive_examples["mol"].tolist() + negative_examples["mol"].tolist()
    
    class_label = positive_examples["Class"].tolist() + negative_examples["Class"].tolist()
    #convert 1 to "Yes" and 0 to "No"" in class_label
    #class_label = ["1" if i == 1 else "0" for i in class_label]
    data_examples = list(zip(smiles, class_label))
    return data_examples

# scaffold sampling

def top_k_scaffold_similar_molecules(target_smiles, _data, k):
    #drop the target_smiles from the dataset
    _data = _data[_data["mol"] != target_smiles]
    molecule_smiles_list = _data['mol'].tolist()
    label_list = _data['Class'].tolist()
    #label_list = ["1" if i == 1 else "0" for i in label_list]

    target_mol = Chem.MolFromSmiles(target_smiles)
    if target_mol is not None:
        target_scaffold = MurckoScaffold.GetScaffoldForMol(target_mol)
    else:
        print("Error: Unable to create a molecule from the provided SMILES string.")
        #drop the target_smiles from the dataset
        return None

    target_fp = rdMolDescriptors.GetMorganFingerprint(target_scaffold, 2)
    RDLogger.DisableLog('rdApp.warning')
    warnings.filterwarnings("ignore", category=UserWarning)
    similarities = []
    
    for i,smiles in enumerate(molecule_smiles_list):
        mol = Chem.MolFromSmiles(smiles)
        try:
            scaffold = MurckoScaffold.GetScaffoldForMol(mol)
            scaffold_fp = rdMolDescriptors.GetMorganFingerprint(scaffold, 2)
            tanimoto_similarity = DataStructs.TanimotoSimilarity(target_fp, scaffold_fp)
            # print(tanimoto_similarity)
            similarities.append((smiles, tanimoto_similarity,label_list[i]))
        except:
            continue
    similarities.sort(key=lambda x: x[1], reverse=True)
    top_similar_molecules = similarities[:k]
    return top_similar_molecules

def create_pred_prompt(input_smiles,pp_examples):
    prompt = "You are an expert chemist, your task is to predict the property of molecule using your experienced chemical property prediction knowledge.\nPlease strictly follow the format, no other information can be provided. Given the SMILES string of a molecule, predict the molecular properties of a given drug SMILES string, predict whether it blocks (1) or not blocks (0). Consider factors such as molecular weight, atom count,Veber's Rule, Physicochemical Properties, bond types, and functional groups,Graph-Based Descriptors, morgan descriptors in order to assess the compound's drug-likeness and its potential to serve as an effective therapeutic agent,please answer with only 1 or 0. A few examples are provided in the beginning.\n"
    for example in pp_examples:
        prompt += f"SMILES: {example[0]}\n Inhibit: {example[-1]}\n"
    prompt += f"SMILES: {input_smiles}\nInhibit:\n"
    return prompt

Few-shot In-context Learning 

In [ ]:

from tdc.single_pred import ADME
from tdc.single_pred import Tox
## CYP3A4_Substrate_CarbonMangels
data = ADME(name = 'BBB_Martins')
split = data.get_split()
name ='bbb' #hERG
#data = Tox(name = name)
split = data.get_split()
bbb_train = split['train']
bbb_test = split['test']
bbb_test.rename(columns={'Y': 'Class'}, inplace=True)
bbb_test.rename(columns={'Drug': 'mol'}, inplace=True)
bbb_train.rename(columns={'Y': 'Class'}, inplace=True)
bbb_train.rename(columns={'Drug': 'mol'}, inplace=True)


def generate_response_by_gpt4(prompt, model_engine = "gpt-4o"):
    completion = client.chat.completions.create(
        model=model_engine, temperature=1, n=5, 
        messages=[{"role": "user", "content": prompt}],
    )

    message = completion.choices
    message = [i.message.content.strip() for i in message]
    return message

def radom_sample_examples(data,sample_size):
    positive_examples = data[data["Class"] == 1].sample(int(sample_size/2))
    negative_examples = data[data["Class"] == 0].sample(int(sample_size/2))
    smiles = positive_examples["mol"].tolist() + negative_examples["mol"].tolist()
    
    class_label = positive_examples["Class"].tolist() + negative_examples["Class"].tolist()
    #convert 1 to "Yes" and 0 to "No"" in class_label
    #class_label = ["1" if i == 1 else "0" for i in class_label]
    data_examples = list(zip(smiles, class_label))
    return data_examples

# scaffold sampling

def top_k_scaffold_similar_molecules(target_smiles, _data, k):
    #drop the target_smiles from the dataset
    _data = _data[_data["mol"] != target_smiles]
    molecule_smiles_list = _data['mol'].tolist()
    label_list = _data['Class'].tolist()
    #label_list = ["1" if i == 1 else "0" for i in label_list]

    target_mol = Chem.MolFromSmiles(target_smiles)
    if target_mol is not None:
        target_scaffold = MurckoScaffold.GetScaffoldForMol(target_mol)
    else:
        print("Error: Unable to create a molecule from the provided SMILES string.")
        #drop the target_smiles from the dataset
        return None

    target_fp = rdMolDescriptors.GetMorganFingerprint(target_scaffold, 2)
    RDLogger.DisableLog('rdApp.warning')
    warnings.filterwarnings("ignore", category=UserWarning)
    similarities = []
    
    for i,smiles in enumerate(molecule_smiles_list):
        mol = Chem.MolFromSmiles(smiles)
        try:
            scaffold = MurckoScaffold.GetScaffoldForMol(mol)
            scaffold_fp = rdMolDescriptors.GetMorganFingerprint(scaffold, 2)
            tanimoto_similarity = DataStructs.TanimotoSimilarity(target_fp, scaffold_fp)
            # print(tanimoto_similarity)
            similarities.append((smiles, tanimoto_similarity,label_list[i]))
        except:
            continue
    similarities.sort(key=lambda x: x[1], reverse=True)
    top_similar_molecules = similarities[:k]
    return top_similar_molecules

def create_pred_prompt(input_smiles,pp_examples):
    prompt = "You are an expert chemist, your task is to predict the property of molecule using your experienced chemical property prediction knowledge.\nPlease strictly follow the format, no other information can be provided. Given the SMILES string of a molecule, predict the molecular properties of a given drug SMILES string, predict whether it blocks (1) or not blocks (0). Consider factors such as molecular weight, atom count,Veber's Rule, Physicochemical Properties, bond types, and functional groups,Graph-Based Descriptors, morgan descriptors in order to assess the compound's drug-likeness and its potential to serve as an effective therapeutic agent,please answer with only 1 or 0. A few examples are provided in the beginning.\n"
    for example in pp_examples:
        prompt += f"SMILES: {example[0]}\n Inhibit: {example[-1]}\n"
    prompt += f"SMILES: {input_smiles}\nInhibit:\n"
    return prompt

model_engine = ['gpt-4o']
sample_nums = [100]
sample_methods = ['scaffold']
detail_save_folder = '' # path to save the generated result 
paras = 0
for sample_method in sample_methods:
    for sample_num in sample_nums:
        for model in model_engine:
            if paras < 0:
                paras += 1
                continue
            detail_predict_file = detail_save_folder + 'test_{}_{}_{}_{}.csv'.format(name, model, sample_num, sample_method)
            log_file = detail_save_folder + 'test_{}_{}_{}_{}.log'.format(name, model, sample_num, sample_method)
            
            if os.path.exists(detail_predict_file):
                detail_results = pd.read_csv(detail_predict_file)
                #convert the column to list
                detail_results = detail_results.values.tolist()
            else:
                detail_results = []
                
                
            now = datetime.datetime.now()
            date_time_str = now.strftime("%Y-%m-%d %H:%M:%S")
            with open(log_file, "a") as file:
                file.write("=" * 30 + date_time_str + "=" * 30 + "\n")
                
            if sample_method == 'random':
                para_index = 0 
                mol_examples = radom_sample_examples(bbb_train,sample_num)
                for i in tqdm(range(0, len(bbb_test))):
                    if para_index < 0:
                        para_index += 1
                        continue
                    example = [(bbb_test.iloc[i]['mol'],bbb_test.iloc[i]['Class'])]
                    pred_y = []
                    generated_results = []
                    for text in example:
                        prompt = create_pred_prompt(text[0], mol_examples)
                        with open(log_file, "a") as file:
                            file.write(prompt + "\n")
                            file.write("=" * 50 + "\n")
                        if model == 'gpt-3.5-turbo':
                            generated_p = generate_response_by_gpt35(prompt)
                        elif model == 'gpt-4o':
                            generated_p = generate_response_by_gpt4(prompt)
                        generated_results.append(generated_p)
                        detail_results.append([text[0]] + [text[-1]] + generated_p)
                        
                        print('training iterations',i)
                        if (i+1) % 10 == 0:
                            details_df = pd.DataFrame(detail_results, columns=['smiles', 'class_label', 'pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5'])
                            details_df.to_csv(detail_predict_file, index=False)
                            print('save file')
                details_df = pd.DataFrame(detail_results, columns=['smiles', 'class_label', 'pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5'])
                details_df.to_csv(detail_predict_file, index=False)
            
            elif sample_method == 'scaffold':
                para_index = 0
                for i in tqdm(range(0, len(bbb_test))):
                    example = [(bbb_test.iloc[i]['mol'],bbb_test.iloc[i]['Class'])]
                    pred_y = []
                    generated_results = []
                    # print(para_index)
                    if para_index < 0:
                        para_index += 1
                        continue
                    for text in example:
                        mol_examples = top_k_scaffold_similar_molecules(text[0], bbb_train, sample_num)
                        prompt = create_pred_prompt(text[0],mol_examples)
                        # print(prompt)
                        with open(log_file, "a") as file:
                            file.write(prompt + "\n")
                            file.write("=" * 50 + "\n")
                        if model == 'gpt-3.5-turbo':
                            generated_p = generate_response_by_gpt35(prompt)
                        elif model == 'gpt-4o':
                            generated_p = generate_response_by_gpt4(prompt)                
                        generated_results.append(generated_p)
                        detail_results.append([text[0]] + [text[-1]] + generated_p)
                        
                        print('training iterations',i)
                        if (i+1) % 10 == 0:
                            details_df = pd.DataFrame(detail_results, columns=['smiles', 'class_label', 'pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5'])
                            details_df.to_csv(detail_predict_file, index=False)
                details_df = pd.DataFrame(detail_results, columns=['smiles', 'class_label', 'pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5'])
                details_df.to_csv(detail_predict_file, index=False)

  1%|          | 1/131 [00:01<04:16,  1.98s/it]

training iterations 0


  2%|▏         | 2/131 [00:03<03:07,  1.45s/it]

training iterations 1


  2%|▏         | 3/131 [00:03<02:35,  1.21s/it]

...

training iterations 128


 99%|█████████▉| 130/131 [03:12<00:01,  1.23s/it]

training iterations 129


100%|██████████| 131/131 [03:13<00:00,  1.48s/it]

training iterations 130


In [24]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score
# name ='DILI'
# sample_methods = ['scaffold']
# sample_nums = [100]
model_engine = ['gpt-4o']
detail_save_folder =''
# Function to calculate metrics
def calculate_metrics(y_true, y_pred):
    """
    Calculate Accuracy, Precision, Recall, ROC AUC, and F1 Score.
    """
    metrics = {}
    metrics['Accuracy'] = accuracy_score(y_true, y_pred)
    metrics['Precision'] = precision_score(y_true, y_pred, zero_division=0)
    metrics['Recall'] = recall_score(y_true, y_pred, zero_division=0)
    metrics['ROC AUC'] = roc_auc_score(y_true, y_pred)
    metrics['F1 Score'] = f1_score(y_true, y_pred)
    return metrics

# Iterate over sample methods and calculate metrics
for sample_method in sample_methods:
    for sample_num in sample_nums:
        for model in model_engine:
            detail_predict_file = detail_save_folder + 'test_{}_{}_{}_{}.csv'.format(name, model, sample_num, sample_method)
            metrics_file = detail_save_folder + 'metrics_{}_{}_{}_{}.txt'.format(name, model, sample_num, sample_method)
            if os.path.exists(detail_predict_file):
                # Load the DataFrame
                details_df = pd.read_csv(detail_predict_file)
                print(details_df.head(5))
                df = details_df.replace({"Yes": 1, "No": 0})
                df[['class_label', 'pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5']] = df[['class_label', 'pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5']].apply(pd.to_numeric)
                # Open the metrics file for writing
                with open(metrics_file, 'w') as f:
                    f.write(f"Metrics for {detail_predict_file}\n")
                    f.write("=" * 50 + "\n")

                    # Iterate over prediction columns
                    for pred_col in ['pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5']:
                        f.write(f"Metrics for {pred_col}:\n")

                        # Calculate metrics
                        y_true = details_df['class_label']
                        y_pred = details_df[pred_col]
                        metrics = calculate_metrics(y_true, y_pred)

                        # Write metrics to the file
                        for metric_name, metric_value in metrics.items():
                            f.write(f"{metric_name}: {metric_value:.4f}\n")
                        f.write("\n")

                    f.write("=" * 50 + "\n")
                print(f"Metrics saved to {metrics_file}")

                                              smiles  class_label  pred_1  \
0  Cc1c(O)cccc1C(=O)N[C@@H](CSc1ccccc1)[C@H](O)C[...          1.0     1.0   
1  CN(CCN(C)S(=O)(=O)c1ccc(NS(C)(=O)=O)cc1)c1nc2c...          1.0     1.0   
2  C=C1C[C@]23C[C@H]4[C@H]5[C@]67C[C@@H](OC(C)=O)...          0.0     0.0   
3  CCOc1cc(N)c(Cl)cc1C(=O)NC[C@@H]1C[NH+](Cc2ccc(...          1.0     1.0   
4           COc1ccc(CNC(=O)c2cc3cc(Cl)ccc3o2)c(OC)c1          1.0     0.0   

   pred_2  pred_3  pred_4  pred_5  
0     0.0     0.0     1.0     0.0  
1     1.0     1.0     1.0     1.0  
2     0.0     1.0     1.0     0.0  
3     1.0     1.0     1.0     1.0  
4     0.0     0.0     0.0     0.0  
Metrics saved to metrics_hERG_gpt-4o_100_scaffold.txt
